In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import os
import glob
import json
import csv
import sys
import numpy as np
import math
import functools
from tqdm import tqdm

In [ ]:
!ls /hot/data/flights/*.csv | head

In [ ]:
df = pd.read_csv('/hot/data/flights/flights_on_time_performance_2009_01.csv', nrows=10)
cleaned_columns = list(map(lambda x: x.lower(), list(df.columns)))

In [4]:
df.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_AIRLINE_ID,OP_CARRIER,TAIL_NUM,...,DIV4_TAIL_NUM,DIV5_AIRPORT,DIV5_AIRPORT_ID,DIV5_AIRPORT_SEQ_ID,DIV5_WHEELS_ON,DIV5_TOTAL_GTIME,DIV5_LONGEST_GTIME,DIV5_WHEELS_OFF,DIV5_TAIL_NUM,Unnamed: 109
0,2009,1,1,18,7,2009-01-18,9E,20363,9E,87099E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009,1,1,19,1,2009-01-19,9E,20363,9E,87459E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009,1,1,20,2,2009-01-20,9E,20363,9E,89249E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009,1,1,21,3,2009-01-21,9E,20363,9E,80259E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009,1,1,22,4,2009-01-22,9E,20363,9E,89239E,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
output_columns = ['CarrierName', 'CarrierCode', 'FlightNumber',
                               'Day', 'Month', 'Year', 'DayOfWeek',
                               'OriginCity', 'OriginState', 'OriginAirportIATACode', 'OriginLongitude', 'OriginLatitude',
                               'OriginAltitude',
                               'DestCity', 'DestState', 'DestAirportIATACode', 'DestLongitude', 'DestLatitude', 'DestAltitude',
                               'Distance',
                               'CancellationReason', 'Cancelled', 'Diverted', 'CrsArrTime', 'CrsDepTime',
                               'ActualElapsedTime', 'AirTime', 'ArrDelay',
                               'CarrierDelay', 'CrsElapsedTime',
                               'DepDelay', 'LateAircraftDelay', 'NasDelay',
                               'SecurityDelay', 'TaxiIn', 'TaxiOut', 'WeatherDelay',
                               'AirlineYearFounded', 'AirlineYearDefunct']

In [6]:
Sfinal = set(map(lambda x: x.lower(), output_columns))

In [7]:
Sinput = set(map(lambda x: x.replace('_', ''), cleaned_columns))

In [8]:
csvinput_cols = Sinput & Sfinal

In [9]:
# indices
csvinput_cols

{'actualelapsedtime',
 'airtime',
 'arrdelay',
 'cancelled',
 'carrierdelay',
 'crsarrtime',
 'crsdeptime',
 'crselapsedtime',
 'dayofweek',
 'depdelay',
 'distance',
 'diverted',
 'lateaircraftdelay',
 'month',
 'nasdelay',
 'securitydelay',
 'taxiin',
 'taxiout',
 'weatherdelay',
 'year'}

In [10]:
normalcols = [name.replace('_', '') for name in cleaned_columns]

In [11]:
relevant_column_pairs = sorted([(name, normalcols.index(name)) for name in csvinput_cols], key=lambda x: x[1])

In [12]:
relevant_columns = [t[0] for t in relevant_column_pairs]
relevant_indices = [t[1] for t in relevant_column_pairs]
relevant_columns, relevant_indices

(['year',
  'month',
  'dayofweek',
  'crsdeptime',
  'depdelay',
  'taxiout',
  'taxiin',
  'crsarrtime',
  'arrdelay',
  'cancelled',
  'diverted',
  'crselapsedtime',
  'actualelapsedtime',
  'airtime',
  'distance',
  'carrierdelay',
  'weatherdelay',
  'nasdelay',
  'securitydelay',
  'lateaircraftdelay'],
 [0, 2, 4, 29, 31, 36, 39, 40, 42, 47, 49, 50, 51, 52, 54, 56, 57, 58, 59, 60])

In [13]:
# should parse delayed? --> do so if representation is SMALLER than smallest optimized representation that's possible.
# for each column compute statistics

# needs df, df_str
def get_stats(df, df_str):
    stats = []
    for c in df.columns:
        stat = {'name' : c, 'type': str(df[c].dtype)}

        null_count = df[c].isna().count()
        stat['nullable'] = null_count > 0
        stat['nrows'] = len(df[c])
        stat['nnulls'] = null_count
        stat['cardinality'] = len(df[c].unique())

        # csv data from str dataframe!
        stat['csvcellmin'] = df_str[c].str.len().min()
        stat['csvcellmax'] = df_str[c].str.len().max()

        if stat['type'] == 'int64':
            # check min/max
            stat['imin'] = int(df[c].min())
            stat['imax'] = int(df[c].max())

        if stat['type'] == 'float64':
            stat['fmin'] = df[c].min()
            stat['fmax'] = df[c].max()

        stats.append(stat)
    return pd.DataFrame(stats)


In [14]:
optimization_list = ['null_value_optimization', 'i8_range_compression',
                     'i16_range_compression',
                     'i24_range_compression',
                    'range_compression_optimization',
                    'delay_parsing',
                    'dict_singlebyte_encoding',
                    'dict_encoding']

# now for each column, decide which optimization is possible!
def determine_col_optimizations(row, threshold=0.05):
    # NVO?
    # null values form more than 95% of all rows or less than 5% of all optimizations
    null_ratio = row['nnulls'] / row['nrows']
    null_optimization = null_ratio <= threshold or null_ratio >= 1.0 - threshold
    
    opts = {'null_value_optimization' : null_optimization}
    
    # integer range squeeze?
    if not math.isnan(row['imin']) and not math.isnan(row['imax']):
        imax_range = int(row['imax']) - int(row['imin'])
    
        # can we compress to 4 bytes, 2 bytes, 1 byte?
        if imax_range < 2**8:
            opts['i8_range_compression'] = True
        if imax_range < 2**16:
            opts['i16_range_compression'] = True
        if imax_range < 2**24:
            opts['i24_range_compression'] = True
        if imax_range < 2**32:
            opts['i32_range_compression'] = True
            opts['range_compression_optimization'] = True
            
    # check whether cell size is < 7 bytes -> delay parsing!
    # use 7 here to reserve 1 byte for length/flags
    csv_max_cell_size = row['csvcellmax'] - row['csvcellmin'] + 1
    opts['delay_parsing'] = csv_max_cell_size <= 7
        
    # dictionary encoding
    card = row['cardinality']
    max_dict_size = 2 ** 16
    if card < 2**8:
        opts['dict_singlebyte_encoding'] = True
    if card <= max_dict_size:
        opts['dict_encoding'] = True
    
    #raise Exception()
    return opts
    

In [17]:
# function to determine how many columns can do each optimization (no details, simple analysis)
def detect_file_specific_optimizations(path, relevant_indices=None, threshold=0.05):
    df = pd.read_csv(path, usecols=relevant_indices, low_memory=False)
    df_str = pd.read_csv(path, dtype=str, usecols=relevant_indices, low_memory=False)
    
    df_stats = get_stats(df, df_str)
    df_stats['optimizations'] = df_stats.apply(functools.partial(determine_col_optimizations), axis=1)
    
    # for each column count how many of each optimizations are available
    df_this_file_opt = pd.DataFrame(list(df_stats['optimizations'].values))
    row = df_this_file_opt.sum().to_dict()
    row['path'] = path
    row['ncolumns'] = len(df.columns)

    return row

In [16]:
path = '/hot/data/flights/flights_on_time_performance_2009_01.csv'
row = detect_file_specific_optimizations(path)

/home/lspiegel/.local/lib/python3.6/site-packages/ipython-7.13.0-py3.6.egg/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (77,84) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [19]:
# run detection over all files (this might take a while!!!)
paths = glob.glob('/hot/data/flights/flights_on_time_performance_*.csv')

In [ ]:
rows = []
for path in tqdm(paths):
    row = detect_file_specific_optimizations(path)
    rows.append(row)

 32%|███▏      | 42/132 [57:29<1:56:24, 77.61s/it]

In [18]:
row

{'null_value_optimization': 110,
 'i8_range_compression': 11,
 'i16_range_compression': 19,
 'i24_range_compression': 21,
 'i32_range_compression': 21,
 'range_compression_optimization': 21,
 'delay_parsing': 81,
 'dict_singlebyte_encoding': 74,
 'dict_encoding': 110,
 'path': '/hot/data/flights/flights_on_time_performance_2009_01.csv',
 'ncolumns': 110}

Plot figure out of specialization across files

In [ ]:
# plot how many files could have optimizations!
name = os.path.basename(path)

names = ['FULLY SPECIALIZED'] + list(df_opt.path.apply(os.path.basename).values)
n_names = len(names)

n_opts = len(optimization_list)

count_matrix = np.zeros((n_names, n_opts)) * np.nan
n_columns = df_opt.iloc[0]['ncolumns']


# fill in first row with ncolumns
count_matrix[0, :] = n_columns

# fill in from rows
for idx, row in df_opt.iterrows():
    row = row.to_dict()
    for key, value in row.items():
        if key in optimization_list:
            j = optimization_list.index(key)
            count_matrix[idx + 1, j] = value
count_matrix.shape

In [ ]:
import copy
fig, ax = plt.subplots()

min_val, max_val = 0, len(df.columns)

cmap = copy.copy(plt.cm.get_cmap("Blues"))
cmap.set_bad('gray',1.)
ax.matshow(count_matrix, cmap=cmap)

for i in range(count_matrix.shape[1]):
    for j in range(count_matrix.shape[0]):
        c = count_matrix[j,i]
        if not math.isnan(c):
            txt = ax.text(i, j, str(int(c)), va='center', ha='center', fontsize=28)
            # cf. https://osxastrotricks.wordpress.com/2014/12/02/add-border-around-text-with-matplotlib/
            txt.set_path_effects([path_effects.Stroke(linewidth=2, foreground='white'),
                       path_effects.Normal()])
# ticks are names of files

ax = plt.gca();

# Major ticks
ax.set_xticks(np.arange(0, n_opts, 1))
ax.set_yticks(np.arange(0, n_names, 1))

# Labels for major ticks
# ax.set_xticklabels(np.arange(1, n_opts + 1, 1))
ax.set_xticklabels(optimization_list, rotation = 90)
# ax.set_yticklabels(np.arange(1, n_names + 1, 1))
ax.set_yticklabels(names)

# Minor ticks
ax.set_xticks(np.arange(-.5, n_opts, 1), minor=True)
ax.set_yticks(np.arange(-.5, n_names, 1), minor=True)

# Gridlines based on minor ticks
ax.grid(which='minor', color='w', linestyle='-', linewidth=2)

In [ ]:
names

In [ ]:
masked_array = np.ma.array (a, mask=np.isnan(a))
cmap = matplotlib.cm.jet
cmap.set_bad('white',1.)
ax.imshow(masked_array, interpolation='nearest', cmap=cmap)

In [ ]:
fig, ax = plt.subplots()

min_val, max_val = 0, 15

intersection_matrix = np.random.randint(0, 10, size=(max_val, max_val))

ax.matshow(intersection_matrix, cmap=plt.cm.Blues)

for i in range(15):
    for j in range(15):
        c = intersection_matrix[j,i]
        ax.text(i, j, str(c), va='center', ha='center')

In [ ]:
# filter specialization for each file!!!

# could write benchmark with two numbers -> parse, then specialize!